In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Input, Dense,Embedding, Concatenate, TimeDistributed
from tensorflow.keras.models import Model,load_model, model_from_json
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

2021-11-18 13:57:46.309049: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-11-18 13:57:46.309078: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Read the Data and Create DF

In [5]:
en_text = open('data/europarl-v7.pl-en.en', 'r').read().split('\n')
pl_text = open('data/europarl-v7.pl-en.pl', 'r').read().split('\n')
print(len(pl_text), len(en_text))

632566 632566


In [6]:
language_df = pd.DataFrame(columns=['en', 'pl'])
en_list = []
pl_list = []
for i in range(len(pl_text)):
    en_list.append(en_text[i])
    pl_list.append(pl_text[i])
language_df['en'] = en_list
language_df['pl'] = pl_list
language_df.head()

,en,pl
0,Action taken on Parliament's resolutions: see ...,Działania podjęte w wyniku rezolucji Parlament...
1,Documents received: see Minutes,Składanie dokumentów: patrz protokół
2,Written statements (Rule 116): see Minutes,Oświadczenia pisemne (art. 116 Regulaminu): pa...
3,Texts of agreements forwarded by the Council: ...,Teksty porozumień przekazane przez Radę: patrz...
4,Membership of Parliament: see Minutes,Skład Parlamentu: patrz protokół
